In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./data/", one_hot=False)

E:\Anconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/train-labels-idx1-ubyte.gz
Extracting ./data/t10k-images-idx3-ubyte.gz
Extracting ./data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
num_classes = 10
num_hidden = 128
drop_rate = 0.3
learning_rate = 0.001
batch_size = 128
num_steps = 8300

def multilayer_perceptron(x,reuse=True,is_training=False):
    with tf.variable_scope("multilayer_perceptron",reuse=reuse):
        output_layer_1 = tf.layers.dense(inputs=x,units=num_hidden)
        output_layer_1 = tf.layers.dropout(output_layer_1,rate=drop_rate,training=is_training)
        
        output_layer_2 = tf.layers.dense(inputs=output_layer_1,units=num_classes)
        
        return output_layer_2

def model_fn(features,labels,mode):
    x = features["images"]
    
    logit_train = multilayer_perceptron(x,reuse=False,is_training=True)
    logit_test = multilayer_perceptron(x,reuse=True,is_training=False)
    
    pre_class = tf.argmax(logit_test,1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode,predictions=logit_test)
    
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_train,
                                                                           labels=tf.cast(labels,tf.int32)))
    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op,
                                                              global_step=tf.train.get_global_step())
    acc_op = tf.metrics.accuracy(labels=labels,predictions=pre_class)
    
    esc = tf.estimator.EstimatorSpec(mode,loss=loss_op,train_op=train_op,eval_metric_ops={"accuracy":acc_op},predictions=pre_class)
    
    return esc


In [3]:
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\卓卓\\AppData\\Local\\Temp\\tmp0n1bal0y', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FB23684470>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\卓卓\AppData\Local\Temp\tmp0n1bal0y\model.ckpt.
INFO:tensorflow:loss = 2.3970351, step = 1
INFO:tensorflow:global_step/sec: 618.932
INFO:tensorflow:loss = 0.44930625, step = 101 (0.163 sec)
INFO:tensorflow:global_step/sec: 721.352
INFO:tensorflow:loss = 0.35778984, step = 201 (0.139 sec)
INFO:tensorflow:global_step/sec: 748.263
INFO:tensorflow:loss = 0.36686218, step = 301 (0.134 sec)
INFO:tensorflow:global_step/sec: 789.506
INFO:tensorflow:loss = 0.5642871, step = 401 (0.127 sec)
INFO:tensorflow:global_step/sec: 795.77
INFO:tensorflow:loss = 0.29384053, step = 501 (0.126 sec)
INFO:tensorflow:global_step/sec: 771.287
INFO:tensorflow:loss = 0.4426526, step = 601 (0.130 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:global_step/sec: 802.136
INFO:tensorflow:loss = 0.4247058, step = 8001 (0.126 sec)
INFO:tensorflow:global_step/sec: 777.267
INFO:tensorflow:loss = 0.12477614, step = 8101 (0.129 sec)
INFO:tensorflow:global_step/sec: 842.583
INFO:tensorflow:loss = 0.3132099, step = 8201 (0.118 sec)
INFO:tensorflow:Saving checkpoints for 8300 into C:\Users\卓卓\AppData\Local\Temp\tmp0n1bal0y\model.ckpt.
INFO:tensorflow:Loss for final step: 0.14940348.


In [5]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.test.images}, y=mnist.test.labels,
    batch_size=batch_size, shuffle=False)
model.evaluate(input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-01-30-09:49:22
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\卓卓\AppData\Local\Temp\tmp0n1bal0y\model.ckpt-8300
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-01-30-09:49:23
INFO:tensorflow:Saving dict for global step 8300: accuracy = 0.9262, global_step = 8300, loss = 0.29185098
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8300: C:\Users\卓卓\AppData\Local\Temp\tmp0n1bal0y\model.ckpt-8300


{'accuracy': 0.9262, 'loss': 0.29185098, 'global_step': 8300}